### Pre requisites
1. Clone this repo and do pip install -e .
2. Have ragas-experimental installed 
3. DEV RAGAS APP TOKEN
4. Clone git clone https://huggingface.co/datasets/explodinggradients/yann-lecun-wisdom

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls ../yann-lecun-wisdom/

README.md             yann_profile.json
yann-lecun_posts.json yann_test.json


In [2]:
from json import load

with open('../yann-lecun-wisdom/yann_test.json', 'r') as f:
    data = load(f)

print(data)

[{'question': 'What are your views on LLM limitations and their ability to generalize?', 'citations': ['7085498609558843392'], 'grading_notes': "- LLMs can't generalize like humans.\n- AI systems can't yet mimic human understanding.\n- Highlight AI's limitations in text memorization.\n- Current AI lacks adaptability to new contexts."}, {'question': "What are Yann LeCun's thoughts on the future steps in AI, particularly the usefulness and limitations of current LLMs?", 'citations': ['7140695461815820289', '7060629056785895424'], 'grading_notes': "- Current LLMs have limitations but are still useful\n- Discuss the balance between potential AI threats and benefits\n- Outline practicality over perfection in AI\n- Explore realistic AI advancements, not science fiction\n- Encourage discussions on AI's current and future role"}, {'question': 'How can countries like India foster AI innovation and development?', 'citations': ['7267674826373304322'], 'grading_notes': '- Suggest creating industry

In [5]:
from ragas_experimental import BaseModel

class TestDataset(BaseModel):
    question: str
    citations: list[str]
    grading_notes: str

In [ ]:
import os

RAGAS_APP_TOKEN = ""
RAGAS_API_BASE_URL = "https://api.dev.app.ragas.io"

os.environ["RAGAS_APP_TOKEN"] = RAGAS_APP_TOKEN
os.environ["RAGAS_API_BASE_URL"] = RAGAS_API_BASE_URL

In [7]:
from ragas_experimental import Project

# p = Project.create(
#     name="yann-lecun-wisdom",
#     description="Yann LeCun Wisdom",
# )


In [8]:
PROJECT_ID = "1ef0843b-231f-4a2c-b64d-d39bcee9d830"

In [9]:
p = Project(project_id=PROJECT_ID)
p

Project(name='yann-lecun-wisdom')

do you actually need to pass a model here?

In [10]:
# test_dataset = p.create_dataset(name="yann-lecun-wisdom-dataset", model=TestDataset)
test_dataset = p.get_dataset(dataset_id="8572180f-fddf-46c5-b943-e6ff6448eb01", model=TestDataset)
test_dataset

Dataset(name=yann-lecun-wisdom-dataset, model=TestDataset, len=0)

In [11]:
test_dataset.load()

NOTE: here there is a problem: how do you batch upload a test dataset?

In [ ]:
from tqdm import tqdm

for item in data:
    t = TestDataset(question=item["question"], citations=item["citations"], grading_notes=item["grading_notes"])
    # test_dataset.append(t)

In [12]:
# de
from ragas_experimental.llm import ragas_llm
from ragas_experimental.metric import DiscreteMetric
from openai import AsyncOpenAI

llm = ragas_llm(provider="openai",model="gpt-4o",client=AsyncOpenAI())

my_metric = DiscreteMetric(
    llm=llm,
    name='correctness',
    prompt="Given the Question: {query} \n Evaluate if given answer {response} \n based on the Grading notes\n: {grading_notes}.",
    values=["pass","fail"],
)



# test LLM as judge
result = my_metric.score(query="what is your response", response="this is my response",grading_notes="- response should not contains word response")
result

'fail'

In [13]:
from ragas_experimental.metric import MetricResult

In [4]:
from linkedin_ai.main import LinkedinAI

In [14]:
!ls ../yann-lecun-wisdom/

README.md             yann_profile.json
yann-lecun_posts.json yann_test.json


In [15]:
my_ai = await LinkedinAI.from_bm25('../yann-lecun-wisdom/yann-lecun_posts.json')

Loaded 437 LinkedIn posts
BM25 index initialized


In [16]:
await my_ai.ask("what is your response")

2025/04/14 20:57:18 WARNING mlflow.tracing.processor.mlflow: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


"The context provided includes discussions on AI, open source, and the nature of science. Here are some key points:\n\n1. **AI and Open Source**: Yann LeCun emphasizes the importance of AI systems being open source, similar to the infrastructure of the Internet. He argues that AI assistants, which will mediate our digital interactions, should be open and crowd-sourced to ensure they are trustworthy and contain all human culture and knowledge. This is why Meta made Llama-2 open source.\n\n2. **Microsoft's Stance**: There is a critique of Microsoft President Brad Smith's comments, which seem to misrepresent the concept of open source by comparing the ownership structures of Meta and OpenAI. LeCun suggests that Microsoft's stance is reminiscent of its past anti-open source position, despite its current use of Linux in Azure.\n\n3. **Debate on Science**: A debate involving Yann LeCun and Elon Musk is highlighted, focusing on the definition of science. The discussion underscores the importa

In [20]:
class ExperimentModel(TestDataset):
    response: str
    score: str
    score_reason: str

@p.experiment(ExperimentModel)
async def experiment_func(item: TestDataset):
    response = await my_ai.ask(item.question)
    score = await my_metric.ascore(query=item.question, response=response, grading_notes=item.grading_notes)
    return ExperimentModel(question=item.question, citations=item.citations, grading_notes=item.grading_notes, response=response, score=score.result, score_reason=score.reason)

In [21]:
c = await experiment_func(test_dataset[0])
c

ExperimentModel(question='What are your views on LLM limitations and their ability to generalize?', citations=['7085498609558843392'], grading_notes="- LLMs can't generalize like humans.\n- AI systems can't yet mimic human understanding.\n- Highlight AI's limitations in text memorization.\n- Current AI lacks adaptability to new contexts.", response='Based on the provided context, it seems that there is a recognition of the limitations of LLMs (Large Language Models) in terms of their ability to generalize. In Post 2, a quote from Melanie Michell highlights that the assumptions we make for humans, such as the ability to generalize understanding to new situations, are not yet appropriate for AI systems like LLMs. Additionally, in Post 3, there is an expression of opinion on the limitations of LLMs, noting that they lack certain features of intelligent behavior, such as understanding the physical world, having persistent memory, reasoning, and planning. These limitations suggest that whil

In [22]:
await experiment_func.run_async(test_dataset)

100%|██████████| 30/30 [00:07<00:00,  4.27it/s]


Experiment(name=kind_hamilton, model=ExperimentModel)

In [26]:
test_dataset[0]._row_id

'haDDwxhEhWq4'